## Summary

---

## Imports

In [1]:
%env CUDA_VISIBLE_DEVICES=1
%env TF_FORCE_UNIFIED_MEMORY=1
%env XLA_PYTHON_CLIENT_MEM_FRACTION=4.0

env: CUDA_VISIBLE_DEVICES=1
env: TF_FORCE_UNIFIED_MEMORY=1
env: XLA_PYTHON_CLIENT_MEM_FRACTION=4.0


In [2]:
import concurrent.futures
import contextlib
import itertools
import os
import tempfile
import urllib.request
from datetime import datetime
from pathlib import Path

import dotenv
import elaspic2 as el2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns
import torch
from elaspic2.plugins.rosetta_ddg import RosettaDDG
from kmbio import PDB
from kmtools import structure_tools
from tqdm.auto import tqdm

## Parameters

In [3]:
NOTEBOOK_DIR = Path("30_cagi6_mapk3").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_cagi6_mapk3')

In [4]:
UNIPROT_ID = "P27361"

UNIPROT_ID

'P27361'

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [6]:
version = datetime.now().isoformat(timespec="hours")

version

'2021-09-28T12'

## Download data

In [7]:
def download(url, filename):
    urllib.request.urlretrieve(url, filename)

In [8]:
def load_sequence(sequence_file):
    with sequence_file.open("rt") as fin:
        data = fin.read()
    chunks = []
    for line in data.split("\n"):
        if line.startswith(">"):
            continue
        chunks.append(line.strip())
    return "".join(chunks)

In [9]:
if not NOTEBOOK_DIR.joinpath("validation.py").is_file():
    import synapseclient
    import synapseutils

    dotenv.load_dotenv("../.env")
    syn = synapseclient.Synapse()
    syn.login(os.environ["SYNAPSE_USERNAME"], os.environ["SYNAPSE_PASSWORD"])
    _ = synapseutils.syncFromSynapse(syn, "syn26019572", path=NOTEBOOK_DIR)

In [10]:
sequence_file = NOTEBOOK_DIR.joinpath(f"{UNIPROT_ID}.fasta")

if not sequence_file.is_file():
    download(f"https://www.uniprot.org/uniprot/{UNIPROT_ID}.fasta", sequence_file)

sequence = load_sequence(sequence_file)

In [11]:
structure_file = NOTEBOOK_DIR.joinpath(f"{UNIPROT_ID}.pdb")

if not structure_file.is_file():
    download(f"https://alphafold.ebi.ac.uk/files/AF-{UNIPROT_ID}-F1-model_v1.pdb", structure_file)

with structure_file.open("r") as fin:
    structure_blob = fin.read()

In [12]:
alignment_file = NOTEBOOK_DIR.joinpath(f"{UNIPROT_ID}.a3m.gz")

if not alignment_file.is_file():
    from elaspic2.plugins.alphafold import mmseqs2

    dotenv.load_dotenv("../.env")
    with mmseqs2.api_gateway(mmseqs2.MMSEQS2_HOST_URL) as gateway:
        alignment = mmseqs2.run_mmseqs2(sequence, gateway=gateway)
        assert alignment[1] == f"{sequence}\n"
        alignment_df = pd.DataFrame({"alignment": alignment})
        pq.write_table(pa.Table.from_pandas(alignment_df, preserve_index=False), alignment_file)

alignment = pq.read_table(alignment_file).to_pandas()["alignment"].values.tolist()

## Load data

In [13]:
def mutation_matches_sequence(mutation, sequence):
    wt, pos, mut = mutation[0], mutation[1:-1], mutation[-1]
    pos = int(pos)
    return sequence[pos - 1] == wt

In [14]:
def sequence_matches_structure(sequence, structure_blob):
    with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
        with open(tmp_file.name, "wt") as fout:
            fout.write(structure_blob)
        structure = PDB.load(tmp_file.name)

    chain_sequence = structure_tools.get_chain_sequence(
        structure[0]["A"], if_unknown="replace", unknown_residue_marker=""
    )
    return sequence == chain_sequence

In [15]:
result_template_df = pd.read_csv(NOTEBOOK_DIR.joinpath("mapk3_predictions.tsv"), sep="\t")

result_template_df.head(2)

,MAPK3-variant,DDG-NotPO4,Standard-Deviation-DDG-NotPO4,DDG-PO4,Standard-Deviation-DDG-PO4,Ratio-kcat/Km,Standard-Deviation-Ratio-kcat/Km,Comment
0,p.I73M,*,*,*,*,*,*,*
1,p.Q79H,*,*,*,*,*,*,*


In [16]:
result_df = result_template_df.copy()
result_df["mut"] = result_df["MAPK3-variant"].str[2:]

In [17]:
assert all([mutation_matches_sequence(mut, sequence) for mut in result_df["mut"]])

In [18]:
assert sequence_matches_structure(sequence, structure_blob)

In [19]:
assert alignment[1] == f"{sequence}\n"

## Run Rosetta

In [20]:
rosetta_ddg_data = RosettaDDG.build(
    structure_file,
    protocol="cartesian_ddg",
    energy_function="beta_nov16_cart",
    interface=0,
)

In [21]:
def rosetta_ddg_worker(mut, data):
    results = RosettaDDG.analyze_mutation(f"A_{mut}", data)
    results = {"mut": mut} | {f"rosetta_{key}": value for key, value in results.items()}
    return results

In [22]:
rosetta_results_file = NOTEBOOK_DIR.joinpath("rosetta-results.parquet")

if rosetta_results_file.is_file():
    rosetta_results_df = pq.read_table(rosetta_results_file).to_pandas()
else:
    with concurrent.futures.ThreadPoolExecutor(len(result_df)) as pool:
        rosetta_results = list(
            tqdm(
                pool.map(
                    rosetta_ddg_worker,
                    result_df["mut"].values.tolist(),
                    itertools.repeat(rosetta_ddg_data),
                ),
                total=len(result_df),
            )
        )
    rosetta_results_df = pd.DataFrame(rosetta_results)
    pq.write_table(
        pa.Table.from_pandas(rosetta_results_df, preserve_index=False), rosetta_results_file
    )

## Run AlphaFold

In [23]:
def worker(mutation, data):
    try:
        results = AlphaFold.analyze_mutation(f"A_{mutation}", data)
    except Exception as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"alphafold_core_{key}": value for key, value in results.items()}
        return results

In [24]:
alphafold_results_file = NOTEBOOK_DIR.joinpath("alphafold-results.parquet")

if alphafold_results_file.is_file():
    alphafold_results_df = pq.read_table(alphafold_results_file).to_pandas()
else:
    from elaspic2.plugins.alphafold import AlphaFold, AlphaFoldAnalyzeError, AlphaFoldBuildError
    from jax.lib import xla_bridge

    jax_device = xla_bridge.get_backend().platform
    print(f"Device: {jax_device}")

    AlphaFold.load_model(device=jax_device)
    data = AlphaFold.build(sequence, ligand_sequence=None, msa=alignment)

    result_list = []
    for mut in tqdm(result_df["mut"], leave=False):
        result = worker(mut, data)
        result_list.append({"mut": mut} | result)
    alphafold_results_df = pd.DataFrame(result_list)

    pq.write_table(
        pa.Table.from_pandas(alphafold_results_df, preserve_index=False), alphafold_results_file
    )

In [25]:
display(alphafold_results_df.head(2))
print(len(alphafold_results_df))

,mut,alphafold_core_scores_residue_plddt_wt,alphafold_core_scores_protein_plddt_wt,alphafold_core_scores_protein_max_predicted_aligned_error_wt,alphafold_core_scores_proten_ptm_wt,alphafold_core_features_residue_experimentally_resolved_wt,alphafold_core_features_residue_predicted_lddt_wt,alphafold_core_features_residue_msa_first_row_wt,alphafold_core_features_residue_single_wt,alphafold_core_features_residue_structure_module_wt,...,alphafold_core_features_residue_experimentally_resolved_mut,alphafold_core_features_residue_predicted_lddt_mut,alphafold_core_features_residue_msa_first_row_mut,alphafold_core_features_residue_single_mut,alphafold_core_features_residue_structure_module_mut,alphafold_core_features_protein_experimentally_resolved_mut,alphafold_core_features_protein_predicted_lddt_mut,alphafold_core_features_protein_msa_first_row_mut,alphafold_core_features_protein_single_mut,alphafold_core_features_protein_structure_module_mut
0,I73M,89.605976,84.518212,31.75,0.849999,"[3.6290748, 3.7973425, 4.2734876, 3.559348, 4....","[-10.512778, -11.93231, -9.935118, -9.103899, ...","[-2.8592641, 3.4308205, 14.783053, 5.0693607, ...","[60.97126, 55.834232, 1.5938658, -7.575698, -1...","[0.0015794635, 0.011161171, -0.0056965444, 0.0...",...,"[3.5453632, 3.7268736, 4.1792274, 3.458398, 4....","[-10.369872, -11.793799, -9.807745, -8.991145,...","[-1.1854149, 2.4412565, 13.099518, 5.5893917, ...","[54.19502, 53.576317, 6.718537, -12.898656, -1...","[0.0015215352, 0.010927603, -0.005702028, 0.00...","[2.784755, 2.9134758, 3.2904088, 2.779093, 3.3...","[-8.535637, -9.482369, -8.275576, -7.4080825, ...","[4.2533255, 4.472761, -0.6898077, 1.5236061, -...","[26.811329, 10.481789, 7.400012, 4.595044, 5.8...","[0.0011052352, 0.010742479, -0.005919944, 0.00..."
1,Q79H,84.960966,84.518212,31.75,0.849999,"[3.940667, 4.0633817, 4.4250245, 4.0075483, 4....","[-7.2024946, -8.241891, -7.25713, -6.562296, -...","[12.790734, 16.888803, -6.8924904, 4.88217, -2...","[56.02134, -35.143005, -67.70256, 20.18287, 19...","[0.0031776056, 0.010669984, -0.0059339087, 0.0...",...,"[4.253464, 4.3940325, 4.7798467, 4.2907968, 4....","[-7.1870084, -8.175636, -7.200021, -6.506134, ...","[13.19207, 14.478604, -7.066455, 6.3435163, -2...","[51.011642, -34.14663, -64.46977, 16.284336, 6...","[0.0030293465, 0.010777913, -0.005934259, 0.00...","[2.7799199, 2.9081771, 3.2842107, 2.7743576, 3...","[-8.5375595, -9.483548, -8.2759495, -7.408301,...","[4.228886, 4.4589715, -0.700582, 1.5512381, -6...","[26.7827, 10.457484, 7.4125285, 4.6370673, 5.8...","[0.0011114243, 0.010763805, -0.005919809, 0.00..."


12


## Run ProteinSolver

In [26]:
def worker(mutation, data):
    try:
        results = ProteinSolver.analyze_mutation(f"A_{mutation}", data)
    except Exception as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"proteinsolver_core_{key}": value for key, value in results.items()}
        return results

In [27]:
proteinsolver_results_file = NOTEBOOK_DIR.joinpath("proteinsolver-results.parquet")

if proteinsolver_results_file.is_file():
    proteinsolver_results_df = pq.read_table(proteinsolver_results_file).to_pandas()
else:
    from elaspic2.plugins.proteinsolver import ProteinSolver

    ProteinSolver.load_model(device=device)
    
    data = ProteinSolver.build(structure_file, sequence, None, remove_hetatms=False)

    result_list = []
    for mut in tqdm(result_df["mut"], leave=False):
        result = worker(mut, data)
        result_list.append({"mut": mut} | result)
    proteinsolver_results_df = pd.DataFrame(result_list)

    pq.write_table(
        pa.Table.from_pandas(proteinsolver_results_df, preserve_index=False),
        proteinsolver_results_file,
    )

In [28]:
display(proteinsolver_results_df.head(2))
print(len(proteinsolver_results_df))

,mut,proteinsolver_core_score_wt,proteinsolver_core_score_mut,proteinsolver_core_features_residue_wt,proteinsolver_core_features_protein_wt,proteinsolver_core_features_residue_mut,proteinsolver_core_features_protein_mut
0,I73M,0.416913,0.011631,"[-3.84128999710083, 0.8013101816177368, 0.0243...","[-1.3879121541976929, 0.6547006964683533, 0.12...","[-3.6069047451019287, 0.22248061001300812, -0....","[-1.3863307237625122, 0.647650420665741, 0.120..."
1,Q79H,0.044277,0.028519,"[-0.13290949165821075, 0.675199568271637, -0.7...","[-1.3879121541976929, 0.6547006964683533, 0.12...","[-4.47153377532959, 0.4991134703159332, -0.200...","[-1.3995500802993774, 0.6538957357406616, 0.12..."


12


## Run ProtBert

In [29]:
def worker(mutation, data):
    try:
        results = ProtBert.analyze_mutation(f"A_{mutation}", data)
    except Exception as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"protbert_core_{key}": value for key, value in results.items()}
        return results

In [30]:
protbert_results_file = NOTEBOOK_DIR.joinpath("protbert-results.parquet")

if protbert_results_file.is_file():
    protbert_results_df = pq.read_table(protbert_results_file).to_pandas()
else:
    from elaspic2.plugins.protbert import ProtBert

    ProtBert.load_model(device=device)
    data = ProtBert.build(sequence, ligand_sequence=None)

    result_list = []
    for mut in tqdm(result_df["mut"], leave=False):
        result = worker(mut, data)
        result_list.append({"mut": mut} | result)
    protbert_results_df = pd.DataFrame(result_list)

    pq.write_table(
        pa.Table.from_pandas(protbert_results_df, preserve_index=False), protbert_results_file
    )

In [31]:
display(protbert_results_df.head(2))
print(len(protbert_results_df))

,mut,protbert_core_score_wt,protbert_core_score_mut,protbert_core_features_residue_wt,protbert_core_features_protein_wt,protbert_core_features_residue_mut,protbert_core_features_protein_mut
0,I73M,0.978126,0.000716,"[0.13129837810993195, -0.03898658975958824, -0...","[0.07031252980232239, -0.013878961093723774, -...","[0.10953198373317719, -0.06380636990070343, -0...","[0.0704592913389206, -0.012903289869427681, -0..."
1,Q79H,0.991156,0.001122,"[0.1967078298330307, 0.04909868910908699, -0.0...","[0.07031252980232239, -0.013878961093723774, -...","[0.19389745593070984, 0.05682474002242088, -0....","[0.06953047960996628, -0.013008753769099712, -..."


12


## MSA

## Run `ELASPIC2`

### Initialize the `ELASPIC2` model



In [32]:
model = el2.ELASPIC2(device=device)

/opt/conda/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PCA from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Some weights of the model checkpoint at /scratch/strokach/workspace/elaspic2/src/elaspic2/plugins/protbert/data/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are i

### Make predictions

In [33]:
protein_features = model.build(
    structure_file=structure_file,
    protein_sequence=sequence,
    ligand_sequence=None,
    remove_hetatms=True,
)

protein_features

ELASPIC2Data(is_interface=False, protbert_data=ProtBertData(sequence='MAAAAAQGGGGGEPRRTEGVGPGVPGEVEMVKGQPFDVGPRYTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLREIQILLRFRHENVIGIRDILRASTLEAMRDVYIVQDLMETDLYKLLKSQQLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLINTTCDLKICDFGLARIADPEHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINMKARNYLQSLPSKTKVAWAKLFPKSDSKALDLLDRMLTFNPNKRITVEEALAHPYLEQYYDPTDEPVAEEPFTFAMELDDLPKERLKELIFQETARFQPGVLEAP'), proteinsolver_data=Data(edge_attr=[19140, 2], edge_index=[2, 19140], x=[379]))

In [34]:
mutation_features = list(
    tqdm(
        (model.analyze_mutation(mut, protein_features) for mut in result_df["mut"]),
        total=len(result_df),
    )
)

In [35]:
# In all cases, higher scores means less stable (same as ΔΔG)
result_df["protbert_score"] = [
    f["protbert_core_score_wt"] - f["protbert_core_score_mut"] for f in mutation_features
]
result_df["proteinsolver_score"] = [
    f["proteinsolver_core_score_wt"] - f["proteinsolver_core_score_mut"] for f in mutation_features
]
result_df["el2_score"] = model.predict_mutation_effect(mutation_features).tolist()

KeyError: 'pca-proteinsolver_core_features_protein_wt-core'

In [ ]:
# In all cases, higher scores means less stable (same as ΔΔG)
result_df["protbert_ratio"] = [
    f["protbert_core_score_mut"] / f["protbert_core_score_wt"] for f in mutation_features
]
result_df["proteinsolver_ratio"] = [
    f["proteinsolver_core_score_mut"] / f["proteinsolver_core_score_wt"] for f in mutation_features
]
result_df["el2_ratio"] = (0.5 / model.predict_mutation_effect(mutation_features)).tolist()

## Combine

In [ ]:
rosetta_results_df["rosetta_dg_mut"] = (
    rosetta_results_df["rosetta_dg_change"] + rosetta_results_df["rosetta_dg_wt"]
)
assert np.allclose(
    rosetta_results_df["rosetta_dg_change"],
    rosetta_results_df["rosetta_dg_mut"] - rosetta_results_df["rosetta_dg_wt"],
)

rosetta_results_df["rosetta_dg_score"] = rosetta_results_df["rosetta_dg_change"]
rosetta_results_df["rosetta_dg_ratio"] = (
    rosetta_results_df["rosetta_dg_wt"] / rosetta_results_df["rosetta_dg_mut"]
)

In [ ]:
try:
    del result_df["rosetta_dg_score"], result_df["rosetta_dg_ratio"]
except KeyError:
    pass

result_df = result_df.merge(
    rosetta_results_df[["mut", "rosetta_dg_score", "rosetta_dg_ratio"]], on="mut", how="left"
)

## Visualize

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(
    result_df[["protbert_score", "proteinsolver_score", "el2_score", "rosetta_dg_score"]]
)

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(
    result_df[["protbert_ratio", "proteinsolver_ratio", "el2_ratio", "rosetta_dg_ratio"]]
)

## Write results

In [ ]:
output_dir = NOTEBOOK_DIR.joinpath("submission")
output_dir.mkdir(exist_ok=True)

output_dir

In [ ]:
%%file {output_dir}/strokach_desc.md
# Submission for CAGI6—MAPK3 challenge

## Overview

- `strokach_modelnumber_1.tsv` → Predictions made using ELASPIC2 [1].
- `strokach_modelnumber_2.tsv` → Predictions made using ProteinSolver [2].
- `strokach_modelnumber_3.tsv` → Predictions made using ProtBert [3].
- `strokach_modelnumber_4.tsv` → Predictions made using Rosetta's cartesian_ddg protocol.

## References

- [1] Strokach, A., Lu, T.Y., Kim, P.M., 2021. ELASPIC2 (EL2): Combining Contextualized Language Models and Graph Neural Networks to Predict Effects of Mutations. Journal of Molecular Biology, Computation Resources for Molecular Biology 433, 166810. https://doi.org/10.1016/j.jmb.2021.166810
- [2] Strokach, A., Becerra, D., Corbi-Verge, C., Perez-Riba, A., Kim, P.M., 2020. Fast and Flexible Protein Design Using Deep Graph Neural Networks. Cell Systems. https://doi.org/10.1016/j.cels.2020.08.016
- [3] Elnaggar, A., Heinzinger, M., Dallago, C., Rehawi, G., Wang, Y., Jones, L., Gibbs, T., Feher, T., Angerer, C., Steinegger, M., Bhowmik, D., Rost, B., 2020. ProtTrans: Towards Cracking the Language of Life’s Code Through Self-Supervised Deep Learning and High Performance Computing. bioRxiv 2020.07.12.199554. https://doi.org/10.1101/2020.07.12.199554

In [ ]:
result_dfs = {}
for i, metric in enumerate(["el2", "proteinsolver", "protbert", "rosetta_dg"]):
    result_dfs[metric] = result_df.copy()
    result_dfs[metric]["DDG-NotPO4"] = -result_dfs[metric][f"{metric}_score"].values
    result_dfs[metric]["Standard-Deviation-DDG-NotPO4"] = 1.0
    result_dfs[metric]["DDG-PO4"] = -result_dfs[metric][f"{metric}_score"].values
    result_dfs[metric][" Standard-Deviation-DDG-PO4"] = 1.0
    result_dfs[metric]["Ratio-kcat/Km"] = result_dfs[metric][f"{metric}_ratio"].values
    result_dfs[metric]["Standard-Deviation-Ratio-kcat/Km"] = 1.0
    result_dfs[metric]["Comment"] = "*"

    print(metric)
    display(result_dfs[metric].head())

    output_file = output_dir.joinpath(f"strokach_modelnumber_{i + 1}.tsv")
    result_dfs[metric][result_template_df.columns].to_csv(output_file, sep="\t", index=False)
    !python {NOTEBOOK_DIR}/validation.py {output_file}